In [27]:
import torch, utils, math, collections, nmt_utils
from torch import nn
class Seq2SeqEncoder(utils.Encoder):
    """
    用于序列到序列学习的循环神经网络编码器。
    使用了嵌入层(embedding layer)来获得输入序列中每个词元的特征向量。
    嵌入层的权重是一个矩阵,其行数等于输入词表的大小(vocab_size),其列数等于特征向量的维度(embed_size)。
    对于任意输入词元的索引i,嵌入层获取权重矩阵的第i行(从0开始)以返回其特征向量。
    选择了一个多层门控循环单元来实现编码器。
    """
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers, dropout=0, **kwargs):
        super().__init__(**kwargs)
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embed_size) # 嵌入层
        self.rnn = nn.GRU(embed_size, num_hiddens, num_layers, dropout=dropout)

    def forward(self, X, *args):
        X = self.embedding(X) # 输出shape:(batch_size, num_steps, embed_size)
        X = X.permute(1,0,2) # 转置后输出shape:(num_steps, batch_size, embed_size) 第一个轴对应时间步
        output, state = self.rnn(X)
        # 输出output的形状为 (num_steps, batch_size, num_hiddens)
        # 输出state的形状为  (num_layers, batch_size, num_hiddens)
        return output, state # 输出和隐状态 最后一次更新的隐状态即为要传递给Decoder的上下文变量
    
encoder = Seq2SeqEncoder(vocab_size=10, embed_size=8, num_hiddens=16, num_layers=2)
encoder.eval()
X = torch.zeros((4, 7), dtype=torch.long)
print(f"X.shape:{X.shape}")
output, state = encoder(X)
print(f"enc_output.shape:{output.shape}")
print(f"enc_state.shape:{state.shape}")


class Seq2SeqDecoder(utils.Decoder):
    """
    用于序列到序列学习的循环神经网络编码器。
    实现解码器时，我们直接使用编码器最后一个时间步的隐状态来初始化解码器的隐状态。
    这要求使用循环神经网络实现的编码器和解码器具有相同数量的层和隐藏单元。
    为了进一步包含经过编码的输入序列的信息,上下文变量在所有的时间步与解码器的输入进行拼接。
    为了预测输出词元的概率分布，在循环神经网络解码器的最后一层使用全连接层来变换隐状态。
    """
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers, dropout=0, **kwargs):
        super().__init__(**kwargs)
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.GRU(embed_size + num_hiddens, num_hiddens, num_layers, dropout=dropout)
        self.dense = nn.Linear(num_hiddens, vocab_size)

    def init_state(self, encoder_outputs, *args):
        return encoder_outputs[1] # 使用编码器最后一个时间步的隐状态来初始化解码器的隐状态 1表示的是输出(output, state)中的state
    
    def forward(self, X, state):
        X = self.embedding(X).permute(1, 0, 2) # X.shape:(num_steps, batch_size, embed_size)
        context = state[-1].repeat(X.shape[0], 1, 1) # 广播context，使其具有与X相同的num_steps
        print(context.shape)
        X_and_context = torch.concat((X, context), dim=2) # 在embed_size维度上连接输入X和上下文变量state
        output, state = self.rnn(X_and_context, state)
        print(output.shape)
        output = self.dense(output).permute(1, 0, 2)
        print(output.shape)
        # output: (num_steps, batch_size, num_hiddens)
        # state:  (num_layers, batch_size, num_hiddens)
        return output, state
    
decoder = Seq2SeqDecoder(vocab_size=10, embed_size=8, num_hiddens=16, num_layers=2)
decoder.eval()
state = decoder.init_state(encoder(X))
output, state = decoder(X, state)
# output.shape, state.shape

X.shape:torch.Size([4, 7])
enc_output.shape:torch.Size([7, 4, 16])
enc_state.shape:torch.Size([2, 4, 16])
torch.Size([7, 4, 16])
torch.Size([7, 4, 16])
torch.Size([4, 7, 10])


$\tilde{X}$

In [41]:
def sequence_mask(X, valid_len, value=0):
    """
    在序列中将不相关的项(如用于填充的"<pad>")替换为value值的函数
    例如,如果两个序列的有效长度(不包括填充词元)分别为1和2,
    则第一个序列的第一项和第二个序列的前两项之后的剩余项将被清除为value指定的值(默认为0)。
    即:将填充词元的预测排除在损失函数的计算之外
    """
    maxlen = X.size(1)
    mask = torch.arange((maxlen), dtype=torch.float32, device=X.device)[None, :] < valid_len[:, None]
    print(mask)
    X[~mask] = value
    return X
X = torch.ones(2, 3, 4)
sequence_mask(X, torch.tensor([1, 2]), value=0)
print(torch.arange((3), dtype=torch.float32)[None, :])
print(torch.tensor([1,2])[:, None])
print(torch.arange((3), dtype=torch.float32)[None, :]<torch.tensor([1,2])[:, None])

tensor([[ True, False, False],
        [ True,  True, False]])
tensor([[0., 1., 2.]])
tensor([[1],
        [2]])
tensor([[ True, False, False],
        [ True,  True, False]])
